In [1]:
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
import math
import time
from utils.utils import * 
from utils.rf_gaussians import gaussian_2d_curve, gaussian_2d_curve_pol

True


In [2]:
columns = ["x0", "y0", "sigma", "slope", "intercept"]

In [4]:
betas_file = os.path.join(proj_dir, 'rsa', 'roi_analyses' , f'subj06_betas_list_nativesurface_averaged.npy')
betas_mean = np.load(betas_file, allow_pickle=False).astype(np.float32)

In [5]:
betas_mean = betas_mean[~np.isnan(betas_mean).any(axis=1), :] 

In [6]:
np.isnan(betas_mean).any() 


False

In [7]:
# Luis ver1 
params = {'random': True, 
                    'initial': (np.array([-0.4, -0.4, 0.01, 0.1, - 2]), np.array([0.4, 0.4, 2, 10, 2])), 
                    'bounds': (np.array([-1.05, -1.05, 0.01, 1e-08, - np.inf]),np.array([1.05, 1.05, np.inf, np.inf, np.inf])),
                    'loss': 'linear',
                    'method': 'trf'
                   }

initial = params['initial']
bounds = params['bounds']

In [8]:
nsd_dir


'/media/harveylab/STORAGE1_NA/stan-thesis/data/NSD'

In [9]:
sessions = [40, 40, 32, 30, 40, 32, 40, 30]

for i in range(1, 9):
    print(f'GETTING BETAS FOR SUBJ0{i}')
    betas_file = os.path.join(proj_dir, 'rsa', 'roi_analyses' , f'subj0{i}_betas_list_nativesurface_averaged.npy')
    betas = np.load(betas_file, allow_pickle=False).astype(np.float32).T
    if np.isinf(betas).any():
        print(f'FOUND INFS IN subj0{i} s betas')
        continue # jump to next subject
    if np.isnan(betas).any():
        print(f'FOUND NANS in subj0{i} s betas')
        continue
    print(f'Starting fitting for subj0{i}')
    n_betas, n_voxels = betas.shape
    for roi in rois.keys():
        start = time.time()
        mds_file = os.path.join(proj_dir, 'rsa', 'mds', f'subj0{i}', f'subj0{i}_{sessions[i-1]}_{roi}_mds.npy') # need to redo MDS 
        mds = np.load(mds_file, allow_pickle=True).astype(np.float32).T
        print('Check for inf and Nan in array')
        print('------------------------------')
        print(f'Infs in MDS: {np.isinf(mds).any()}, NaNs in MDS: {np.isnan(mds).any()}')
        print('Rotated MDS loaded')
        fit_file = os.path.join(proj_dir, 'fits_rotated', f'subj0{i}')
        if not os.path.exists(fit_file):
            os.makedirs(fit_file)
        fit_file = os.path.join(fit_file, f'fits_subj0{i}_{sessions[i-1]}_trains_version1_{roi}_base.npy')
        if os.path.exists(fit_file):
            print(f'\t\t\ŧskipping {roi}, already exists')
            continue
        fits_roi = pd.DataFrame(columns=columns)
    
        print(f'Starting fitting for ROI {roi}, on version 1')
        for voxel in range(n_voxels):
            if not voxel % 1000:
                print(
                    f'\t\t\t\t\t[{100*voxel/n_voxels:.2f}%] elapsed time since {roi} start: ',
                                f'{time.strftime("%H:%M:%S", time.gmtime(time.time() - start))}'
                            )
            voxel_activity = betas[:, voxel]
            if np.isinf(voxel_activity).any():
                print('There is some inf in voxel_activity')
            if np.isnan(voxel_activity).any():
                print('There is some Nan in voxel_activity')
            if params['random']:
                attempt = 1
                solved = False
                while not solved and attempt <= 10:
                    try:
                        initial_guess = (initial[1] - initial[0]) * np.random.random(initial[0].shape) + initial[0]
                        if np.isinf(initial_guess).any():
                            print('There is some inf in initial_guess')
                        if np.isnan(initial_guess).any():
                            print('There is some Nan in initial_guess')
                        voxel_fit = curve_fit(
                            gaussian_2d_curve_pol,
                            mds,
                            voxel_activity, # check if thats working
                            p0  = initial_guess,
                            bounds=bounds,
                            method = 'trf',
                            ftol = 1e-06
                        )[0]
                        solved = True
                    except RuntimeError:
                        print(f'VOXEL {voxel}: optimal params not found after {attempt} attempts')
                        attempt +1
            
            fits_roi.loc[voxel] = [*voxel_fit]
                
        def gaus_roi(fits):
            return gaussian_2d_curve_pol(mds, *fits)
    
        pred_activity = fits_roi.apply(gaus_roi, axis = 1)
        pred_activity = np.array([np.array(x) for x in pred_activity]).T
        roi_res = np.sum((pred_activity - betas)**2, axis=0)
        roi_tot = sum((betas- np.tile(betas.mean(axis=0), (n_betas, 1)))**2).T
    
        ## Maybe include split here
    
        fits_roi["var_explained"] = 1 - roi_res / roi_tot
        fits_roi["mds_ecc"] = (fits_roi.x0 ** 2 + fits_roi.y0[1] ** 2) ** (1/2)
        fits_roi["mds_ang"] = np.arctan2(fits_roi.x0/bounds[1][0], fits_roi.y0/bounds[1][1])
    
        np.save(fit_file, fits_roi)
        print(f'\t\t\tTime elapsed during {roi}: ',
        f'{time.strftime("%H:%M:%S", time.gmtime(time.time() - start))}'
                    )
    del betas
                                         

GETTING BETAS FOR SUBJ01
Starting fitting for subj01
Check for inf and Nan in array
------------------------------
Infs in MDS: False, NaNs in MDS: False
Rotated MDS loaded
		\ŧskipping V1, already exists
Check for inf and Nan in array
------------------------------
Infs in MDS: False, NaNs in MDS: False
Rotated MDS loaded
		\ŧskipping V2, already exists
Check for inf and Nan in array
------------------------------
Infs in MDS: False, NaNs in MDS: False
Rotated MDS loaded
		\ŧskipping V3, already exists
Check for inf and Nan in array
------------------------------
Infs in MDS: False, NaNs in MDS: False
Rotated MDS loaded
		\ŧskipping hV4, already exists
Check for inf and Nan in array
------------------------------
Infs in MDS: False, NaNs in MDS: False
Rotated MDS loaded
		\ŧskipping VO-1, already exists
Check for inf and Nan in array
------------------------------
Infs in MDS: False, NaNs in MDS: False
Rotated MDS loaded
		\ŧskipping VO-2, already exists
Check for inf and Nan in array